# Load data and split to train and test

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
reviews = pd.read_csv('Reviews.csv')

In [ ]:
reviews.head(10)

In [ ]:
def classifier(row):
    if row['Score'] > 3:
        return 'Positive'
    else:
        return 'Negative'

reviews['Response'] = reviews.apply(classifier, axis=1)

In [ ]:
reviews.head(10)

In [ ]:
total_size = len(reviews)
total_size

In [ ]:
train_size = int(0.70*total_size)
train_size

In [ ]:
trainSet = reviews.head(train_size)
testSet = reviews.tail(total_size-train_size)

In [ ]:
trainSet.Score.value_counts()

In [ ]:
testSet.Score.value_counts()

In [ ]:
trainSet.head(50).Score.value_counts()

In [ ]:
testSet.head(50).Score.value_counts()

# Preprocess texts

In [ ]:
from nltk.tokenize import word_tokenize
import re
import spacy
import nltk
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
def preprocess_text(input_str):
    words = word_tokenize(input_str)
    words = [re.sub(r"[^A-Za-z@]", "", word) for word in words]
    words = [re.sub(r"\S+com", "", word) for word in words]
    words = [re.sub(r"\S+@\S+", "", word) for word in words]
    words = [word for word in words if word!=' ']

    #import nltk stopwords
    stopwords_nltk = nltk.corpus.stopwords.words('english')
    
    #import other lists of stopwords
    with open('stopwords_en.txt', 'r') as f:
        file = f.readlines()
    stopwords = []
    for word in file:
        stopwords.append(word.replace('\n', ''))

    stopwords = stopwords + stopwords_nltk
    stopwords = [s.lower() for s in stopwords]
    
    words = [token.lemma_ for token in nlp(' '.join(words)) if not token.is_punct]
    words = [word.lower() for word in words if word.lower() not in stopwords]
    
    while ' ' in words: words.remove(' ')
    return ' '.join(words)

In [ ]:
train_texts = []
test_texts = []

# remove the [:50] on a more powerful computer
for text in trainSet['Text'].tolist()[:50]:
    train_texts.append(preprocess_text(text))
    
for text in testSet['Text'].tolist()[:50]:
    test_texts.append(preprocess_text(text))

In [ ]:
train_texts

In [ ]:
test_texts

# Vectorise

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vect = CountVectorizer()
x_train_dtm = vect.fit_transform(train_texts)
x_test_dtm = vect.transform(test_texts)

In [ ]:
x_test_dtm

In [ ]:
x_train_dtm

# Build and evaluate model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

Build model

In [ ]:
# train model using x_train_dtm
%time nb.fit(x_train_dtm, trainSet.head(50).Score)

In [ ]:
y_pred_class_nb = nb.predict(x_test_dtm)

Evaluate model

In [ ]:
from sklearn import metrics
metrics.accuracy_score(testSet.head(50).Score, y_pred_class_nb)

In [ ]:
con_metrics_nb = metrics.confusion_matrix(testSet.head(50).Score, y_pred_class_nb)
con_metrics_nb

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(con_metrics_nb, annot=True, fmt='d')
plt.title('Confusion Matrix: Naive Bayes')
plt.show()

In [ ]:
# look at huggingface.com website
pip install transformers

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(testSet.head(50).Score, y_pred_class_nb))

In [ ]:
auc_nb = metrics.roc_auc_score(testSet.head(50).Score, y_pred_class_nb)
print(auc_nb)

In [ ]:
y_pred_class_nb

In [ ]:
reviews.tail(total_size-train_size)

In [ ]:
test_texts

In [ ]:
testSet.head(50).Score

In [ ]:
testSet